In [11]:
import os
import csv
import mmap
import pickle
import numpy as np
import pandas as pd
import datetime as dt
import scipy.io as sio
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
#from pyearth import Earth#conda install sklearn-contrib-py-earth

np.warnings.filterwarnings('ignore') #Runtime warning

In [2]:
#Header of csv: 'Date[0],Time[1],Depth of Airgun(m)[2],Depth of Reciever(m)[3],X Airgun[4],Y Airgun[5],Z Airgun[6],X_R1[7],Y_R1[8],Z_R1[9],SEL1[10],RMS1[11],SEL2[12],RMS2[13],SEL3[14],RMS3[15],SEL4[16],RMS4[17],SEL_full[18],RMS_full[19],T90_1[20],T90_2[21],T90_3[22],T90_4[23],T90_full[24]
def prepData(x_data,y_data,path,skiplist):
    csv_train_path = path
    files = os.listdir(csv_train_path)
    dataTrain = {}
    x_data2 = x_data
    y_data2 = y_data
            
    file = open(skiplist, 'rb',0)
    s = mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ)
    
    for filename in sorted(files):
        if s.find(str.encode(filename)) == -1:
            dirname = csv_train_path + filename
            dataTrain = pd.read_csv(dirname)
            dataTrain = dataTrain.rename(columns={'Depth of Airgun(m)': 'ag_depth','Depth of Reciever(m)': 'hyd_depth'})
            dataTrain = dataTrain[(dataTrain.hyd_depth != -99999)]    
            ag_x = dataTrain['X Airgun']
            ag_y = dataTrain['Y Airgun']
            ag_z = dataTrain['Z Airgun']
            hyd_x = dataTrain['X_R1'] #relative to ag_x
            hyd_y = dataTrain['Y_R1'] #relative to ag_y
            hyd_z = dataTrain['Z_R1'] #relative to ag_z
            r_train = np.sqrt(np.square(hyd_x) + np.square(hyd_y) + np.square(hyd_z))
            logr_train = np.log10(np.sqrt(np.square(hyd_x) + np.square(hyd_y) + np.square(hyd_z)))
            dataTrain = dataTrain.assign(r_train = r_train,logr_train = logr_train,filename = filename)
            dataTrain = dataTrain.dropna(axis=0, how='any')

     #      outliers
            sel_dif = {}
            sel_dif = dataTrain['SEL_full'].diff()
            sel_dif2 = sel_dif.dropna(axis=0, how='any')
            sel_thres = np.abs(np.mean(np.abs(sel_dif2)))
            sel_dif = sel_dif[np.abs(sel_dif.values) < sel_thres]
            dataTrain = dataTrain.loc[sel_dif.index]

     #       rms_dif = {}
     #       rms_dif = dataTrain['RMS_full'].diff()
     #       rms_dif2 = rms_dif.dropna(axis=0, how='any')
     #       rms_thres = np.abs(np.mean(np.abs(rms_dif2)))
     #       rms_dif = rms_dif[np.abs(rms_dif.values) < rms_thres]
     #       dataTrain = dataTrain.loc[rms_dif.index]        

            x_data2 = x_data2.append(dataTrain[['ag_depth', 'hyd_depth','r_train','logr_train']],ignore_index=True)
            y_data2 = y_data2.append(dataTrain[['SEL_full','RMS_full']], ignore_index=True)
     #       data = data.append(dataTrain[['filename','ag_depth', 'hyd_depth','logr_train','r_train','RMS_full','SEL_full']],ignore_index=True)
     #   else:
     #       print('rejected:'+filename)
    return [x_data2,y_data2]

def pickle2(filename,x_data,y_data):
    outfile = open(filename,'wb')
    pickle.dump({'X': x_data, 'SEL': y_data['SEL_full'], 'RMS': y_data['RMS_full']},outfile)
    outfile.close()
    return;

In [20]:
#RMS
#draw from the pickle
infile = open('all_lines(pruned).pickle','rb')
#infile = open('all_lines.pickle','rb')
data = pickle.load(infile, encoding='bytes')
x_train, x_test, y_train, y_test = train_test_split(data['X'],data['RMS'],test_size=0.1,shuffle=True)

In [21]:
print("Models for RMS")
print("=====================Linear=====================")
reg = linear_model.LinearRegression(fit_intercept=True, normalize=False)
# Train the model using the training sets
linreg_model = reg.fit(x_train, y_train)
print('Coefficients (slope): \n', reg.coef_)
print('Intercept: \n', reg.intercept_)

 
y_pred = linreg_model.predict(x_test)

R2 = reg.score(x_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x_test_subset = x_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model.predict(x_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

print("================2nd degree poly================")#2nd degree polynomial
poly = PolynomialFeatures(degree=2)
reg2 = linear_model.LinearRegression(fit_intercept=True, normalize=False)
x2_train = poly.fit_transform(x_train)
x2_test = poly.fit_transform(x_test)

linreg_model2 = reg2.fit(x2_train, y_train)
print('Coefficients (slope): \n', reg2.coef_)
print('Intercept: \n', reg2.intercept_)
 
y_pred = linreg_model2.predict(x2_test)

R2 = reg2.score(x2_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x2_test_subset = x2_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model2.predict(x2_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

print("================3rd degree poly================")#3rd degree polynomial
poly = PolynomialFeatures(degree=3)
reg2 = linear_model.LinearRegression(fit_intercept=True, normalize=False)
x2_train = poly.fit_transform(x_train)
x2_test = poly.fit_transform(x_test)

linreg_model2 = reg2.fit(x2_train, y_train)
print('Coefficients (slope): \n', reg2.coef_)
print('Intercept: \n', reg2.intercept_)
 
y_pred = linreg_model2.predict(x2_test)

R2 = reg2.score(x2_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x2_test_subset = x2_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model2.predict(x2_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

print("================4th degree poly================")#3rd degree polynomial
poly = PolynomialFeatures(degree=4)
reg2 = linear_model.LinearRegression(fit_intercept=True, normalize=False)
x2_train = poly.fit_transform(x_train)
x2_test = poly.fit_transform(x_test)

linreg_model2 = reg2.fit(x2_train, y_train)
print('Coefficients (slope): \n', reg2.coef_)
print('Intercept: \n', reg2.intercept_)
 
y_pred = linreg_model2.predict(x2_test)

R2 = reg2.score(x2_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x2_test_subset = x2_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model2.predict(x2_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

Models for RMS
=====================Linear=====================
Coefficients (slope): 
 [-4.40721687e-03  2.52684045e-03  8.82673606e-05 -9.91289717e+00]
Intercept: 
 197.80367145059915
R^2 score: 0.33 

Total Mean squared error: 28.57 

Subspace Mean squared error: 25.55 

================2nd degree poly================
Coefficients (slope): 
 [ 0.00000000e+00 -4.88418859e-02  3.53177106e-02 -1.03502895e+00
 -1.05502153e+03 -3.46257473e-07  1.31563366e-06 -1.10146147e-06
  1.37139476e-02 -4.01344919e-07  2.24869307e-06 -1.17102881e-02
 -5.91749719e-06  2.47619035e-01  2.48998883e+02]
Intercept: 
 1396.4080175793795
R^2 score: 0.47 

Total Mean squared error: 22.65 

Subspace Mean squared error: 23.11 

================3rd degree poly================
Coefficients (slope): 
 [-9.72350969e-04 -2.86915203e-01  8.14822564e-01 -1.81769218e+02
 -6.19115304e+02 -1.29243317e-06  3.95839472e-05  2.66826700e-04
  2.15132360e-01 -9.33964508e-06 -7.08134371e-04 -7.15706099e-01
  4.09158657e-03  9.

In [22]:
#SEL
#draw from the pickle
infile = open('all_lines(pruned).pickle','rb')
data = pickle.load(infile, encoding='bytes')
x_train, x_test, y_train, y_test = train_test_split(data['X'],data['SEL'],test_size=0.1,shuffle=True)

In [23]:
print("Models for SEL")
print("=====================Linear=====================")
reg = linear_model.LinearRegression(fit_intercept=True, normalize=False)
# Train the model using the training sets
linreg_model = reg.fit(x_train, y_train)
print('Coefficients (slope): \n', reg.coef_)
print('Intercept: \n', reg.intercept_)

 
y_pred = linreg_model.predict(x_test)

R2 = reg.score(x_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x_test_subset = x_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model.predict(x_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

print("================2nd degree poly================")#2nd degree polynomial
poly = PolynomialFeatures(degree=2)
reg2 = linear_model.LinearRegression(fit_intercept=True, normalize=False)
x2_train = poly.fit_transform(x_train)
x2_test = poly.fit_transform(x_test)

linreg_model2 = reg2.fit(x2_train, y_train)
print('Coefficients (slope): \n', reg2.coef_)
print('Intercept: \n', reg2.intercept_)
 
y_pred = linreg_model2.predict(x2_test)

R2 = reg2.score(x2_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x2_test_subset = x2_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model2.predict(x2_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

print("================3rd degree poly================")#3rd degree polynomial
poly = PolynomialFeatures(degree=3)
reg2 = linear_model.LinearRegression(fit_intercept=True, normalize=False)
x2_train = poly.fit_transform(x_train)
x2_test = poly.fit_transform(x_test)

linreg_model2 = reg2.fit(x2_train, y_train)
print('Coefficients (slope): \n', reg2.coef_)
print('Intercept: \n', reg2.intercept_)
 
y_pred = linreg_model2.predict(x2_test)

R2 = reg2.score(x2_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x2_test_subset = x2_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model2.predict(x2_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

print("================4th degree poly================")#4th degree polynomial
poly = PolynomialFeatures(degree=4)
reg2 = linear_model.LinearRegression(fit_intercept=True, normalize=False)
x2_train = poly.fit_transform(x_train)
x2_test = poly.fit_transform(x_test)

linreg_model2 = reg2.fit(x2_train, y_train)
print('Coefficients (slope): \n', reg2.coef_)
print('Intercept: \n', reg2.intercept_)
 
y_pred = linreg_model2.predict(x2_test)

R2 = reg2.score(x2_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x2_test_subset = x2_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model2.predict(x2_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

Models for SEL
=====================Linear=====================
Coefficients (slope): 
 [-2.95018803e-03  1.74091084e-03 -4.59011992e-04 -7.80376214e+00]
Intercept: 
 189.56863817291986
R^2 score: 0.53 

Total Mean squared error: 13.04 

Subspace Mean squared error: 8.67 

================2nd degree poly================
Coefficients (slope): 
 [ 0.00000000e+00 -3.63372597e-02  2.06430080e-02 -5.56530974e-01
 -5.38967785e+02 -7.29162887e-07  9.73798326e-07 -6.87653593e-07
  1.05703807e-02 -3.02057206e-07  1.17940017e-06 -6.59579973e-03
 -3.13140440e-06  1.33170176e-01  1.27784564e+02]
Intercept: 
 797.0475173819473
R^2 score: 0.66 

Total Mean squared error: 9.66 

Subspace Mean squared error: 6.02 

================3rd degree poly================
Coefficients (slope): 
 [-2.57980263e-04  1.68775978e-01  6.34945910e-02 -1.01226700e+02
 -3.45171066e+02 -3.63305165e-05  5.12986185e-05 -1.15661960e-04
 -1.52070574e-01 -5.87990171e-06 -1.37382448e-04 -8.09742657e-02
  2.65946318e-03  5.1126

In [6]:
#this takes a while
#mars = Earth()
#mars.fit(x_train,y_train)
#
#filename = 'mars_sel.pickle'
#outfile = open(filename,'wb')
#
#pickle.dump(mars,outfile)
#outfile.close()

#print("=======Multivariate Adaptive Regression Splines=======")
##https://contrib.scikit-learn.org/py-earth/content.html
#infile = open('mars_sel.pickle','rb')
#mars = pickle.load(infile)
#y_pred = mars.predict(x_test)
#
#R2 = mars.grsq_#MARS score equivalent
#print('R^2 score: %.2f \n' %R2)
#err_total = mean_squared_error(y_test, y_pred)
#print("Total Mean squared error: %.2f \n" %err_total)
# 
#x_test_subset = x_test[100:200]
#y_test_subset = y_test[100:200]
#y_pred_subset = mars.predict(x_test_subset)
#err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
#print("Subspace Mean squared error: %.2f \n" %err_subspace)

#this takes a while

#mars = Earth()
#mars.fit(x_train,y_train)
#
#filename = 'mars_rms.pickle'
#outfile = open(filename,'wb')
#
#pickle.dump(mars,outfile)
#outfile.close()
#

#print("=======Multivariate Adaptive Regression Splines=======")
##https://contrib.scikit-learn.org/py-earth/content.html
#infile = open('mars_rms.pickle','rb')
#mars = pickle.load(infile)
#y_pred = mars.predict(x_test)
#
#R2 = mars.grsq_#MARS score equivalent
#print('R^2 score: %.2f \n' %R2)
#err_total = mean_squared_error(y_test, y_pred)
#print("Total Mean squared error: %.2f \n" %err_total)
# 
#x_test_subset = x_test[100:200]
#y_test_subset = y_test[100:200]
#y_pred_subset = mars.predict(x_test_subset)
#err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
#print("Subspace Mean squared error: %.2f \n" %err_subspace)

=======Multivariate Adaptive Regression Splines=======
R^2 score: 0.66 

Total Mean squared error: 7.85 

Subspace Mean squared error: 5.47 



In [ ]:
#skiplist = 'C:\SPB_Data\pcbenv\skip.txt'#bad data
#
#whole data set
#path1 = r'D:\Machine Learning\NewCSV\CSV1\Line_01_(RMS_and_SEL)\\'
#path2 = r'D:\Machine Learning\NewCSV\CSV1\Line_02_(RMS_and_SEL)\\'
#path3 = r'D:\Machine Learning\NewCSV\CSV1\Line_03_(RMS_and_SEL)\\'
#path4 = r'D:\Machine Learning\NewCSV\CSV1\Line_04_(RMS_and_SEL)\\'
#path5 = r'D:\Machine Learning\NewCSV\CSV1\Line_05_(RMS_and_SEL)\\'
#path6 = r'D:\Machine Learning\NewCSV\CSV1\Line_06_(RMS_and_SEL)\\'
#path7 = r'D:\Machine Learning\NewCSV\CSV1\Line_07_(RMS_and_SEL)\\'
#path8 = r'D:\Machine Learning\NewCSV\CSV1\Line_08_(RMS_and_SEL)\\'
#path9 = r'D:\Machine Learning\NewCSV\CSV1\Line_09_(RMS_and_SEL)\\'
#path10 = r'D:\Machine Learning\NewCSV\CSV1\Line_10_(RMS_and_SEL)\\'
#path11 = r'D:\Machine Learning\NewCSV\CSV1\Line_11_(RMS_and_SEL)\\'
#pathAT = r'D:\Machine Learning\NewCSV\CSV1\Line_AT_(RMS_and_SEL)\\'
#
#[x_data,y_data] = prepData(pd.DataFrame(),pd.Series(),path1,skiplist)
#[x_data,y_data] = prepData(x_data,y_data,path2,skiplist)
#[x_data,y_data] = prepData(x_data,y_data,path3,skiplist)
#[x_data,y_data] = prepData(x_data,y_data,path4,skiplist)
#[x_data,y_data] = prepData(x_data,y_data,path5,skiplist)
#[x_data,y_data] = prepData(x_data,y_data,path6,skiplist)
#[x_data,y_data] = prepData(x_data,y_data,path7,skiplist)
#[x_data,y_data] = prepData(x_data,y_data,path8,skiplist)
#[x_data,y_data] = prepData(x_data,y_data,path9,skiplist)
#[x_data,y_data] = prepData(x_data,y_data,path10,skiplist)
#[x_data,y_data] = prepData(x_data,y_data,path11,skiplist)
#[x_data,y_data] = prepData(x_data,y_data,pathAT,skiplist)
#pickle2('all_lines(pruned,rms).pickle',x_data,y_data)